In [18]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

import numpy as np
from torchvision import models

In [5]:
from google.colab import drive
import torch
from torchvision import datasets, transforms, models
from torch import nn, optim
from torch.utils.data import DataLoader

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Define dataset transformations
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # InceptionV3 requires input size 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/test2', transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Function to initialize the Inception model
def get_model(model_name):
    if model_name == "inception-v3":
        model = models.inception_v3(pretrained=True, aux_logits=True)

        # Freeze all layers
        for param in model.parameters():
            param.requires_grad = False

        # Modify the final fully connected layer for binary classification
        model.fc = nn.Linear(model.fc.in_features, 2)
    return model

In [11]:
# Function to calculate metrics
def calculate_metrics(model, loader, device):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    conf_matrix = confusion_matrix(all_labels, all_predictions)
    TN, FP, FN, TP = conf_matrix.ravel()
    total = conf_matrix.sum()
    accuracy = (TP + TN) / total if total > 0 else 0.0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return accuracy, precision, recall, f1, conf_matrix


In [12]:
# Function to train the model
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5):
    best_val_accuracy = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                if isinstance(outputs, tuple):
                    outputs = outputs[0]
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy

    return best_val_accuracy


In [13]:
# Objective function for hyperparameter optimization
def objective(trial, model_name):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    val_accuracies = []

    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold_idx + 1}/{num_folds}")
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        val_accuracy, _, _, _, _ = calculate_metrics(model, val_loader, device)
        val_accuracies.append(val_accuracy)

    return np.mean(val_accuracies)


In [14]:
# Function to evaluate on the test set
def evaluate_test_set(model_name, best_lr):
    model = get_model(model_name).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    criterion = nn.CrossEntropyLoss()

    fold_metrics = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"\nEvaluating on Fold {fold_idx + 1}/{num_folds}")
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
        fold_metrics.append(calculate_metrics(model, test_loader, device))

    avg_accuracy = np.mean([metrics[0] for metrics in fold_metrics])
    avg_precision = np.mean([metrics[1] for metrics in fold_metrics])
    avg_recall = np.mean([metrics[2] for metrics in fold_metrics])
    avg_f1 = np.mean([metrics[3] for metrics in fold_metrics])
    total_conf_matrix = np.sum([metrics[4] for metrics in fold_metrics], axis=0)

    print("\nAverage Metrics Across Folds:")
    print(f"Accuracy: {avg_accuracy:.2f}, Precision: {avg_precision:.2f}, Recall: {avg_recall:.2f}, F1-Score: {avg_f1:.2f}")
    print(f"Confusion Matrix (sum of all folds):\n{total_conf_matrix}")


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
for model_name in ["inception-v3"]:
    print(f"\nOptimizing for {model_name.upper()}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_name), n_trials=5)
    best_lr = study.best_params['lr']
    print(f"Best Learning Rate for {model_name.upper()}: {best_lr}")
    evaluate_test_set(model_name, best_lr)

[I 2024-12-29 18:16:25,141] A new study created in memory with name: no-name-4db37a96-defd-4f5a-a10c-7a68b6df265e



Optimizing for INCEPTION-V3...


<ipython-input-13-1652910d8ec9>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1/3
Epoch 1/5, Loss: 3.3986012786626816
Epoch 1/5, Validation Accuracy: 66.67%
Epoch 2/5, Loss: 3.140981048345566
Epoch 2/5, Validation Accuracy: 33.33%
Epoch 3/5, Loss: 3.5508705377578735
Epoch 3/5, Validation Accuracy: 63.33%
Epoch 4/5, Loss: 1.9418334066867828
Epoch 4/5, Validation Accuracy: 70.00%
Epoch 5/5, Loss: 1.0411503612995148
Epoch 5/5, Validation Accuracy: 43.33%
Fold 2/3
Epoch 1/5, Loss: 1.1129553020000458
Epoch 1/5, Validation Accuracy: 63.33%
Epoch 2/5, Loss: 0.9388546496629715
Epoch 2/5, Validation Accuracy: 70.00%
Epoch 3/5, Loss: 1.049882486462593
Epoch 3/5, Validation Accuracy: 80.00%
Epoch 4/5, Loss: 0.7679143100976944
Epoch 4/5, Validation Accuracy: 76.67%
Epoch 5/5, Loss: 0.6571283042430878
Epoch 5/5, Validation Accuracy: 73.33%
Fold 3/3
Epoch 1/5, Loss: 0.879697784781456
Epoch 1/5, Validation Accuracy: 76.67%
Epoch 2/5, Loss: 1.027362771332264
Epoch 2/5, Validation Accuracy: 73.33%
Epoch 3/5, Loss: 0.7847142219543457
Epoch 3/5, Validation Accuracy: 80.00%
Ep

[I 2024-12-29 18:32:42,972] Trial 0 finished with value: 0.6777777777777777 and parameters: {'lr': 0.014606192555759987}. Best is trial 0 with value: 0.6777777777777777.
<ipython-input-13-1652910d8ec9>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weig

Fold 1/3
Epoch 1/5, Loss: 0.7407654523849487
Epoch 1/5, Validation Accuracy: 36.67%
Epoch 2/5, Loss: 0.711314469575882
